In [4]:
import pandas as pd
from mootdx.quotes import Quotes

# 创建 Quotes 对象
client = Quotes.factory(market='std')

# 获取 510300 的近一周 1 分钟 K 线数据
# 这里假设一周是5个交易日，每天240个1分钟K线数据，总共是1200条数据
symbol = '510300'
data = client.bars(symbol=symbol, frequency=8, offset=1200)

# 转换为 DataFrame
df = pd.DataFrame(data)
df['datetime'] = pd.to_datetime(df['datetime'])

df


,open,close,high,low,vol,amount,year,month,day,hour,minute,datetime,volume
datetime,,,,,,,,,,,,,
2024-07-03 13:41:00,3.491,3.490,3.492,3.490,514500.0,1796059.0,2024,7,3,13,41,2024-07-03 13:41:00,514500.0
2024-07-03 13:42:00,3.491,3.494,3.494,3.491,1819800.0,6356160.0,2024,7,3,13,42,2024-07-03 13:42:00,1819800.0
2024-07-03 13:43:00,3.494,3.497,3.497,3.494,5264400.0,18404580.0,2024,7,3,13,43,2024-07-03 13:43:00,5264400.0
2024-07-03 13:44:00,3.496,3.495,3.496,3.494,1776200.0,6208677.0,2024,7,3,13,44,2024-07-03 13:44:00,1776200.0
2024-07-03 13:45:00,3.494,3.492,3.495,3.492,1681500.0,5872733.0,2024,7,3,13,45,2024-07-03 13:45:00,1681500.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-08 14:56:00,3.435,3.435,3.435,3.434,28133900.0,96635200.0,2024,7,8,14,56,2024-07-08 14:56:00,28133900.0
2024-07-08 14:57:00,3.435,3.436,3.436,3.434,27483400.0,94404160.0,2024,7,8,14,57,2024-07-08 14:57:00,27483400.0
2024-07-08 14:58:00,3.435,3.434,3.435,3.433,5569500.0,19127328.0,2024,7,8,14,58,2024-07-08 14:58:00,5569500.0


In [5]:
# 历史分时行情，可用
from mootdx.quotes import Quotes

client = Quotes.factory(market='std')
# data=client.minutes(symbol='000001', date='20100105')
# data=client.minutes(symbol='510300', date='20160108')
data=client.minutes(symbol='161226', date='20240110')
data

,price,vol,volume
0,7.46,6540,6540
1,7.48,1870,1870
2,7.46,982,982
3,7.46,308,308
4,7.46,3032,3032
...,...,...,...
235,7.45,362,362
236,7.45,97,97
237,7.45,0,0
238,7.45,0,0


In [24]:
from mootdx.quotes import Quotes
import pandas as pd

# from QHDB.clickhouse.db import storage_client

client = Quotes.factory(market='std')
ret_list=[]
ret=None
code='159980'
index = 0
while True:
    data = client.bars(symbol=code, frequency=4, start=index * 800, offset=(1 + index) * 800)
    # data = client.bars(symbol='000776', frequency=4, start=index * 800, offset=(1 + index) * 800)

    # print(data.to_string())
    # # 前复权
    # data = client.bars(symbol='600036', adjust='qfq')
    #
    # # 后复权
    # data = client.bars(symbol='600036', adjust='hfq')
    ret_list.append(data)
    if data.empty or len(data) < 800:
        break
    index += 1

if len(ret_list)>1:
    # 使用concat函数拼接DataFrame
    result_df = pd.concat(ret_list)

    # print(result_df)

    # 按照索引（datetime）重新排序
    result_df = result_df.sort_index()
    result_df.rename(columns={'datetime': 'date'}, inplace=True)
    # result_df.set_index('date', inplace=True)
    result_df["date"]=pd.to_datetime(result_df["date"])
    selected_columns = ['open', 'close', 'high', 'low', 'volume', 'amount','date',"code"]
    result_df["code"]=code
    result_df = result_df.loc[:, selected_columns]
    # print(result_df.to_string())
    ret=result_df
else:
    # 按照索引（datetime）重新排序
    ret_list[0] = ret_list[0].sort_index()
    ret_list[0].rename(columns={'datetime': 'date'}, inplace=True)
    # ret_list[0].set_index('date', inplace=True)
    ret_list[0]["date"]=pd.to_datetime(ret_list[0]["date"])
    selected_columns = ['open', 'close', 'high', 'low', 'volume', 'amount','date',"code"]
    ret_list[0]["code"]=code
    result_df = ret_list[0].loc[:, selected_columns]
    # print(ret_list[0].to_string())
    ret=ret_list[0]

rr=[]
for i in range((len(ret)-1)//95+1):
    end=len(ret) if (i+1)*95>(len(ret)-1) else (i+1)*95
    # storage_client.insert_dataframe('INSERT INTO tdx.bars_1d VALUES', ret[i*95:end])
    rr.append( ret[i*95:end])

rrr=pd.concat(rr)
# print(len(ret),len(rrr))
rrr

,open,close,high,low,volume,amount,date,code
datetime,,,,,,,,
2019-12-24 15:00:00,1.013,1.016,1.019,1.009,43977212.0,44473124.0,2019-12-24 15:00:00,159980
2019-12-25 15:00:00,1.011,1.017,1.017,1.011,19269988.0,19555762.0,2019-12-25 15:00:00,159980
2019-12-26 15:00:00,1.014,1.014,1.015,1.012,6663502.0,6757245.0,2019-12-26 15:00:00,159980
2019-12-27 15:00:00,1.015,1.016,1.018,1.013,15659583.0,15889983.0,2019-12-27 15:00:00,159980
2019-12-30 15:00:00,1.015,1.014,1.016,1.011,5374263.0,5446320.0,2019-12-30 15:00:00,159980
...,...,...,...,...,...,...,...,...
2024-07-02 15:00:00,1.741,1.748,1.750,1.741,32571300.0,56893488.0,2024-07-02 15:00:00,159980
2024-07-03 15:00:00,1.747,1.763,1.763,1.743,40450400.0,70847480.0,2024-07-03 15:00:00,159980
2024-07-04 15:00:00,1.775,1.766,1.775,1.760,41862900.0,74018392.0,2024-07-04 15:00:00,159980


In [26]:
rrr_save=rrr[['date','open','close','high','low','volume','amount']]
rrr_save.drop(['amount'],axis=1,inplace=True)
rrr_save.rename(columns={'date':'datetime'},inplace=True)
rrr_save

C:\Users\Administrator\AppData\Local\Temp\ipykernel_18452\1281640297.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rrr_save.drop(['amount'],axis=1,inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_18452\1281640297.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rrr_save.rename(columns={'date':'datetime'},inplace=True)


,datetime,open,close,high,low,volume
datetime,,,,,,
2019-12-24 15:00:00,2019-12-24 15:00:00,1.013,1.016,1.019,1.009,43977212.0
2019-12-25 15:00:00,2019-12-25 15:00:00,1.011,1.017,1.017,1.011,19269988.0
2019-12-26 15:00:00,2019-12-26 15:00:00,1.014,1.014,1.015,1.012,6663502.0
2019-12-27 15:00:00,2019-12-27 15:00:00,1.015,1.016,1.018,1.013,15659583.0
2019-12-30 15:00:00,2019-12-30 15:00:00,1.015,1.014,1.016,1.011,5374263.0
...,...,...,...,...,...,...
2024-07-02 15:00:00,2024-07-02 15:00:00,1.741,1.748,1.750,1.741,32571300.0
2024-07-03 15:00:00,2024-07-03 15:00:00,1.747,1.763,1.763,1.743,40450400.0
2024-07-04 15:00:00,2024-07-04 15:00:00,1.775,1.766,1.775,1.760,41862900.0


In [27]:
rrr_save.to_csv('159980-day.csv',index=False)